In [16]:
import boto3

TABLE = boto3.session.Session(region_name='sa-east-1', profile_name='sofie').resource('dynamodb').Table('table_sofier_ledger')
params = dict()

FINAL = list()

while True:
    response = TABLE.scan(**params)
    
    for each in response.get('Items'):
        if each['phase'] == 'BLOCKED' and each['reward'] > 0 and not each['history'][2]['time_stamp'].startswith('2019-11-25'):
            FINAL.append((each['sofier'], each['execution_id'], each['reward'], each['history'][2]['time_stamp']))
    
    last_key = response.get('LastEvaluatedKey')
    if not last_key:
        break
        
    params['ExclusiveStartKey'] = last_key

from pprint import pprint
pprint(FINAL)

[('robertinhooj99@gmail.com',
  '434632f0-0170-11ea-a4c0-b20e75b9452b',
  Decimal('10'),
  '2019-11-14T22:16:13.630245'),
 ('robertinhooj99@gmail.com',
  '889bb352-fff9-11e9-8bc4-fa4859a6e9f3',
  Decimal('10'),
  '2019-11-14T22:16:14.319076'),
 ('rf.rodriguees@gmail.com',
  '0ae5ef40-ca6b-11e9-8e0a-02f385236ef5',
  Decimal('10'),
  '2019-10-04T20:00:00.000000'),
 ('rf.rodriguees@gmail.com',
  '37739f98-c8d7-11e9-ab99-ce583e3945f6',
  Decimal('10'),
  '2019-10-04T20:00:00.000000'),
 ('rf.rodriguees@gmail.com',
  '4ea412b2-ca6f-11e9-a29e-66d3b5e3169a',
  Decimal('10'),
  '2019-10-04T20:00:00.000000'),
 ('rf.rodriguees@gmail.com',
  '6646db76-ca64-11e9-97be-da1355064cfe',
  Decimal('10'),
  '2019-10-04T20:00:00.000000'),
 ('rf.rodriguees@gmail.com',
  '6d356b3c-c8e8-11e9-aaa3-0e85efb10c75',
  Decimal('10'),
  '2019-10-04T20:00:00.000000'),
 ('rf.rodriguees@gmail.com',
  '789109ac-ca67-11e9-8e0a-02f385236ef5',
  Decimal('10'),
  '2019-10-04T20:00:00.000000'),
 ('rf.rodriguees@gmail.com',
 

In [17]:
from datetime import datetime

for sofier, execution_id, reward, time_stamp in FINAL:
    TABLE.update_item(
        Key={'sofier': sofier, 'execution_id': execution_id},
        UpdateExpression='set phase = :ph, last_move = :lm, history = list_append(history, :h)',
        ExpressionAttributeValues={
            ':ph': 'AVAILABLE',
            ':lm': time_stamp,
            ':h': [{'phase': 'AVAILABLE', 'time_stamp': time_stamp}]
        }
    )    